In [79]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle

In [80]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [81]:
course_df.head(2)

,course_id,course_name,course_price,teacher_id,teacher_intro,groups,sub_groups,groups+subgroups,topics,course_published_at_local,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,1800,61888e7bb7fe1c0006850eff,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",NaN,"<img src=""https://images.api.hahow.in/images/6...",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️ 雖然課程當中會帶到相關彩妝技巧，不...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,100,54d5a079065a7e0e00725abe,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計_平面設計,藝術_繪畫與插畫","Illustrator/以拉,配色技巧",42090.98472,"<img src=""https://storage.googleapis.com/hahow...",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


# 課程 Vectorize

In [82]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)


728

In [83]:
def clean_data(x):
    return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'topics', 'will_learn', 'recommended_background', 'target_group', 'required_tools']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(5)

,course_name,teacher_intro,groups,sub_groups,topics,will_learn,recommended_background,target_group,required_tools
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️雖然課程當中會帶到相關彩妝技巧，不過...,熱愛彩妝的人,所需工具為：視課程實際會用到的彩妝用品
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧,"AdobeIllustrator（必備）,camera"
2,數位拼貼的手感,自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、插畫製作等，偶爾...,"藝術,設計","平面設計,電腦繪圖",photoshop,藉由這堂課，可以學習到如何利用身邊的物件自製素材，了解去背、Photoshop的基本操作，並...,,繪圖軟體初學者；喜歡隨性創作；常天馬行空；喜歡剪剪貼貼或收集東西，卻不知該如何收藏。,軟體：AdobePhotoshop、AdobeIllustrator；\n設備：電腦、相機、...
3,Line的貼圖自己動手做！,我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物設計、AppUI設計，201...,"藝術,設計","平面設計,應用設計,電腦繪圖","LINE貼圖設計,photoshop",如果上完課之後好好練習(不懂的地方隨時提問)，就可以畫出你在一般市面上看的到的line貼圖(...,只要手可以拿得起鉛筆和握滑鼠就可以,喜歡畫畫，但是沒有學過電腦繪圖的人、想學基本的電腦繪圖但不知如何入手的人，從10歲-100歲...,Photoshop(可至官網下載試用版)
4,為申請學校或工作寫好英文自傳,在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研細胞奈米科學與皮膚再生醫學的過...,"職場技能,語言","求職,英文","商用英文,履歷撰寫","Apersonalstatementthatyoucanbeproudof,andarewa...",HowtouseGoogleDoc.BasicEnglishlisteningcompreh...,Peopleapplyingforcollege/post-graduatestudieso...,GoogleDocwillbeusedtosubmit/shareyourassignmen...


In [84]:
#import jieba.analyse
# 想說可用jieba之類的，但不一定會比較好
def create_combination(x):
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group
    
    return text

fillna['combination'] = fillna.apply(create_combination, axis=1)
fillna['combination'].head(5)

0    少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新...
1    幾何圖形分割X色塊組合 從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現...
2    數位拼貼的手感 自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、...
3    Line的貼圖自己動手做！ 我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物...
4    為申請學校或工作寫好英文自傳 在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研...
Name: combination, dtype: object

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix

In [86]:
tfidf = TfidfVectorizer(stop_words= 'english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(fillna['combination'])

#print(tfidf.vocabulary_)
tfidf_matrix_nd = tfidf_matrix.toarray()
np.save('tfidf_embedding/item_embeddings_english.npy', tfidf_matrix_nd)

In [87]:
print(len(tfidf.get_feature_names_out()))
list(tfidf.vocabulary_.items())[:2]

23700


[('少女人妻華麗變身', 9959), ('七大妝容七彩的夢幻樂園', 2014)]

In [88]:
from scipy.sparse import csr_matrix
item_embed_FM = np.load('tfidf_embedding/item_embeddings_english.npy')
item_embed_FM = item_embed_FM[:728]
print(tfidf_matrix.shape)
print(item_embed_FM.shape)

(728, 23700)
(728, 23700)


In [89]:
concatenate_item_embed_nd = np.concatenate([tfidf_matrix_nd, item_embed_FM/50], axis = 1)
print(concatenate_item_embed_nd.shape)
concatenate_item_embed = csr_matrix(concatenate_item_embed_nd)

(728, 47400)


In [90]:

# Compute the cosine similarity matrix
print(concatenate_item_embed_nd.shape)
cosine_sim2 = linear_kernel(concatenate_item_embed, concatenate_item_embed)
#Output the shape of tfidf_matrix
cosine_sim2.shape

(728, 47400)


(728, 728)

In [91]:
import pickle
tfidf_save = {"TfidfVectorizer": tfidf, "Tfidf_matrix": tfidf_matrix, "lightFM_embeding_matrix": item_embed_FM, "concatenate_matrix": concatenate_item_embed}
with open('content_embed/all_english.pickle', 'wb') as f:
    pickle.dump(tfidf_save, f)

# According to course similarity to Predict Subgroups

In [92]:
subgroups_df = pd.read_csv('./data/subgroups.csv')
subgroup_num = len(subgroups_df)

subgroups2idx = {}
for (id, name) in zip(subgroups_df["subgroup_id"], subgroups_df["subgroup_name"]):
    subgroups2idx.update({name: id})
idx2subgroups = {v : k for k, v in subgroups2idx.items()}

In [93]:
course2subgroups = {}
#i=0
for (course_id, sub_groups) in zip(course_df["course_id"], course_df["sub_groups"]):
    #i=i+1
    #print(i)
    if pd.isnull(sub_groups):
        course2subgroups.update({course_id: [0]})
    else:
        course2subgroups.update({course_id: [ subgroups2idx[sub_group] for sub_group in sub_groups.split(',')]})

list(course2subgroups.items())[:5]

[('61888e868f154b000781b45a', [1, 2]),
 ('54d5a117065a7e0e00725ac0', [3, 4]),
 ('54d5d9952246e60a009ec571', [3, 5]),
 ('54d7148a2246e60a009ec588', [3, 6, 5]),
 ('5513e92b38239d10005778e1', [7, 8])]

In [94]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_subgroups = []
    # Get the scores of the top most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_subgroups) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                course_id = course_df['course_id'][sim_scores[i][0]]
                for subgroup in course2subgroups[course_id]:
                    if subgroup not in recommend_subgroups:
                        recommend_subgroups.append(subgroup)

    return recommend_subgroups

test = ['61888e868f154b000781b45a']
print(course_df['course_name'].iloc[[course2id_mapping[course] for course in test]].tolist())  
test_predict = get_recommendations_new(test, cosine_sim2, 10)
[ idx2subgroups[subgroup_id] for subgroup_id in test_predict ]

['少女人妻華麗變身：七大妝容七彩的夢幻樂園']


['更多生活品味',
 '護膚保養與化妝',
 '更多職場技能',
 '個人品牌經營',
 '表演藝術',
 '手作小物',
 '更多手作',
 '心靈成長與教育',
 '運動',
 '親子教育']

In [95]:
seen_predict_df = pd.read_csv('./data/test_seen_group.csv')
predict_user = seen_predict_df["user_id"].to_list()
print(len(predict_user))


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

7205
59737


# Predict and Save

In [96]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("subgroup_predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "subgroup"])
    for user_id in predict_users:
        recommend_subgroups_str = [ str(x) for x in get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50)]
        recommend = " ".join(recommend_subgroups_str)
        writer.writerow([user_id, recommend])

# Map50

In [97]:
val_seen_group_df["subgroup"].isna().sum()

33

In [98]:
train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_answer_subgroups = {}
val_seen_group_df_fillna = val_seen_group_df.fillna("")
for seen_user_id, subgroup_ids in zip(val_seen_group_df_fillna["user_id"], val_seen_group_df_fillna["subgroup"]):
    if len(subgroup_ids) > 0:
        course_id_list = [ int(x) for x in subgroup_ids.split(' ')]
        val_answer_subgroups[seen_user_id] = val_answer_subgroups.setdefault(seen_user_id, []) + course_id_list
list(val_answer_subgroups.items())[:5]

[('56dae2b74e3ef90900b7bd0e', [37, 71]),
 ('60e66f29be3e3b0006c4db75', [1, 14, 15]),
 ('5c919efb728ddf00208b9b2b', [1, 3, 5, 25, 29, 50, 51, 52, 66, 67, 72, 73]),
 ('5ac115507997a2001e7c3617', [33, 34, 38, 39]),
 ('5f53b84440c5be3bb873a9d3', [30])]

In [99]:
import ml_metrics

answers = []
predictions = []
map50s = []
for user_id in val_seen_group_df["user_id"]:
    prediction = get_recommendations_new(train_user_haved_purchased_course[user_id], cosine_sim2, top = 50)

    prediction_idxs = prediction
    if user_id in val_answer_subgroups.keys():
        answer_idxs = val_answer_subgroups[user_id]
    else:
        answer_idxs = []

    predictions.append(prediction_idxs)
    answers.append(answer_idxs)
    map50s.append(ml_metrics.mapk(predicted= [prediction_idxs], actual= [answer_idxs], k = 50))
    
print(np.mean(map50s))      
map50 = ml_metrics.mapk(predicted= predictions, actual= answers, k = 50)
map50

0.26766006438937334


0.26766006438937334

In [100]:
#None: 0.264229
#English: 0.26766006438937334

SyntaxError: illegal target for annotation (359312418.py, line 1)